In [ ]:
# Import necessary libraries
import sys
from pathlib import Path
repo_root = Path("/home/ubuntu/michael/MSc-Machine-Learning-Project")
src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
import time
import os
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import pyarrow.feather as feather
import matplotlib.pyplot as plt
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader
from itertools import product 
from mamba_ssm import Mamba
from prob_mamba.utils import arma_fit_forecast, garch_fit_forecast, create_sequences, make_loaders, num_epochs, run_experiment_block_param_validated, param_count
from prob_mamba.eval import build_seq2seq_loaders, prob_head_params, build_prob_mamba_budget_fixed_n, train_prob_mamba_with_history, plot_nll_history, eval_prob_rmse_qlike_laststep
from prob_mamba.models import ProbabilisticMamba

In [ ]:
# Data directory
data_dir = repo_root / "Datasets" / "Processed"

#Set seed for reproducibility
seed = 21003415
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

## Models to test:
1. Econometric baseline ($\text{ARMA}$ for mean, $\text{GARCH }(1,1)$ for volatility)
2. Simple RNN
3. Vanilla Mamba
4. Prob_mamba

In [ ]:
# Econometric baseline on NYSE data
train_df = feather.read_feather(data_dir / "cleaned_NYSE_train.feather")
val_df   = feather.read_feather(data_dir / "cleaned_NYSE_val.feather")
test_df  = feather.read_feather(data_dir / "cleaned_NYSE_test.feather")
# 1. ARMA mean
order, arma_rmse, mu_fc, arma_fit = arma_fit_forecast(
    train_df, val_df, test_df, target_col='y_next', return_fit=True
)

# 2 GARCH variance on ARMA residuals
g_rmse, g_qlike, g_nll, mu_hat, var_hat = garch_fit_forecast(
    train_df, val_df, test_df, target_col='y_next',
    dist='normal', scale=100.0,
    mu_trval=arma_fit.fittedvalues,  # train+val fitted mean
    mu_test=mu_fc                    # test mean forecasts
)

# Results
print(f"Econometric baseline on NYSE data: RMSE={arma_rmse:.6f}, QLIKE={g_qlike:.6f}, NLL={g_nll:.6f}")
resid = pd.Series(arma_fit.resid, dtype="float64").dropna().reset_index(drop=True)
plt.plot(resid.values)
plt.title("ARMA residuals on NYSE data")
plt.show()

In [ ]:
# Econometric baseline on IXIC data
train_df = feather.read_feather(data_dir / "cleaned_IXIC_train.feather")
val_df   = feather.read_feather(data_dir / "cleaned_IXIC_val.feather")
test_df  = feather.read_feather(data_dir / "cleaned_IXIC_test.feather")
# 1. ARMA mean
order, arma_rmse, mu_fc, arma_fit = arma_fit_forecast(
    train_df, val_df, test_df, target_col='y_next', return_fit=True
)

# 2 GARCH variance on ARMA residuals
g_rmse, g_qlike, g_nll, mu_hat, var_hat = garch_fit_forecast(
    train_df, val_df, test_df, target_col='y_next',
    dist='normal', scale=100.0,
    mu_trval=arma_fit.fittedvalues,  # train+val fitted mean
    mu_test=mu_fc                    # test mean forecasts
)

# Results
print(f"Econometric baseline on IXIC data: RMSE={arma_rmse:.6f}, QLIKE={g_qlike:.6f}, NLL={g_nll:.6f}")
resid = pd.Series(arma_fit.resid, dtype="float64").dropna().reset_index(drop=True)
plt.plot(resid.values)
plt.title("ARMA residuals on IXIC data")
plt.show()

In [ ]:
# Econometric baseline on BTC data
train_df = feather.read_feather(data_dir / "BTC_USDT-5m_train.feather")
val_df   = feather.read_feather(data_dir / "BTC_USDT-5m_val.feather")
test_df  = feather.read_feather(data_dir / "BTC_USDT-5m_test.feather")
# 1. ARMA mean
order, arma_rmse, mu_fc, arma_fit = arma_fit_forecast(
    train_df, val_df, test_df, target_col='y_next', return_fit=True
)

# 2 GARCH variance on ARMA residuals
g_rmse, g_qlike, g_nll, mu_hat, var_hat = garch_fit_forecast(
    train_df, val_df, test_df, target_col='y_next',
    dist='normal', scale=1000,
    mu_trval=arma_fit.fittedvalues,  # train+val fitted mean
    mu_test=mu_fc                    # test mean forecasts
)

# Results
print(f"Econometric baseline on BTC data: RMSE={arma_rmse:.6f}, QLIKE={g_qlike:.6f}, NLL={g_nll:.6f}")
resid = pd.Series(arma_fit.resid, dtype="float64").dropna().reset_index(drop=True)
plt.plot(resid.values)
plt.title("ARMA residuals on BTC data")
plt.show()

In [ ]:
if __name__ == "__main__":
    # Configuation
    sequence_length_map = {
        "BTC_USDT-5m": 288,
        "cleaned_IXIC": 270,
        "cleaned_NYSE": 270
    }
    target_column = 'y_next' # next-period log return
    # Data to process
    Arrow_files_prefix= [
        'cleaned_IXIC',
        'cleaned_NYSE',
        'BTC_USDT-5m'
    ]
    # Initialise a dictionary to hold all the final data
    all_processed_data = {}

    for prefix in Arrow_files_prefix:
        print(f"\n Creating Sequences for {prefix}:")

        # Step 1:  Load original data to calculate log returns
        original_path = f"/home/ubuntu/michael/MSc-Machine-Learning-Project/Datasets/Processed/{prefix}.feather"
        original_df = feather.read_feather(original_path)
        date_col = "date" if "date" in original_df.columns else "Date"
        original_df[date_col] = pd.to_datetime(original_df[date_col])
        original_df = original_df.sort_values(date_col).reset_index(drop=True)
        price_col = "Price" if "Price" in original_df.columns else "close"
        if price_col not in original_df.columns:
            raise ValueError("Expected column 'Price' or 'close' in dataframe.")
        # sanity: no non-positive prices
        if (original_df[price_col] <= 0).any():
            raise ValueError(f"Non-positive prices found in {price_col} — clean data first.")
        #Compute log_price, log_returns
        original_df["_log_price"] = np.log(original_df[price_col])
        original_df["ret_t"] = original_df["_log_price"].diff() # today's log return
        original_df["y_next"] = original_df["_log_price"].shift(-1) - original_df["_log_price"] # next period log return
        original_df = original_df.dropna(subset=["y_next"]).reset_index(drop=True)
        print("Computed log_price, ret_t and y_next inb original data.")

        # Step 2: Load the pre-processed data
        train_df = feather.read_feather(f'{data_dir}/{prefix}_train.feather')
        val_df = feather.read_feather(f'{data_dir}/{prefix}_val.feather')
        test_df = feather.read_feather(f'{data_dir}/{prefix}_test.feather')
        for df in [train_df, val_df, test_df]:
            df[date_col] = pd.to_datetime(df[date_col])
        print("Loaded pre-processed taining, validation, and test data")

        # Step 3: Define features to prevent data leakage
        columns_to_drop = ['Date', 'Name', price_col, '_log_price', "y_next"]
        feature_columns = [col for col in train_df.columns if col not in columns_to_drop and col != date_col]
        print(f"Using {len(feature_columns)} input features.")

        # Step 4: Set sequence length
        default_sequence_length = 270
        short_prefix = prefix.replace("cleaned_", "")
        seq_len = sequence_length_map.get(prefix, sequence_length_map.get(short_prefix, default_sequence_length))

        # Step 5: Create windowed sequence 
        X_train, y_train = create_sequences (train_df, seq_len, feature_columns, target_column)
        X_val, y_val = create_sequences(val_df, seq_len, feature_columns, target_column)
        X_test, y_test = create_sequences(test_df, seq_len, feature_columns, target_column)

        # Diagnostic prints
        print(f"---Shapes for {prefix}---")
        print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
        print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
        print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")    

        dataset_name = prefix.split('_')[-1]
        all_processed_data[dataset_name] = {
            'X_train': X_train,
            'y_train': y_train,
            'X_val': X_val,
            'y_val': y_val,
            'X_test': X_test,
            'y_test': y_test
        }
        print("\nAll datasets processed.")

In [ ]:
loaders = {}
for dataset_name, data in all_processed_data.items():
    print(f"\nCreating DataLoaders for {dataset_name}:")
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    X_test = data['X_test']
    y_test = data['y_test']

    train_loader, val_loader, test_loader = make_loaders(
        X_train, y_train, X_val, y_val, X_test, y_test,
        batch=64
    )
    loaders[dataset_name] = {
        'train_loader': train_loader,
        'val_loader': val_loader,
        'test_loader': test_loader
    }
    print("DataLoaders created.")

In [ ]:
budgets = (100_000, 300_000)
seeds   = (0, 1, 2)
val_ep  = max(10, num_epochs // 5)

final_tables = {}
final_details = {}

for dataset_name in all_processed_data.keys():
    print(f"\n### Param validation + training: {dataset_name}")
    train_loader, val_loader, test_loader = loaders[dataset_name]

    # wrap into the expected structure for the runner
    apd = {
        dataset_name: {
            "X_train": all_processed_data[dataset_name]["X_train"],
            "y_train": all_processed_data[dataset_name]["y_train"],
            "X_val":   all_processed_data[dataset_name]["X_val"],
            "y_val":   all_processed_data[dataset_name]["y_val"],
            "X_test":  all_processed_data[dataset_name]["X_test"],
            "y_test":  all_processed_data[dataset_name]["y_test"],
        }
    }

    df_summary, details = run_experiment_block_param_validated(
        dataset_name=dataset_name,
        all_processed_data=apd,
        budgets=budgets,
        rnn_layers_options=[1, 2, 3], 
        mamba_layers_options=[1, 2, 3], 
        seeds=seeds,
        val_epochs=val_ep,
        tol_print=6
    )
    final_tables[dataset_name] = df_summary
    final_details[dataset_name] = details

In [ ]:
# Prob_mamba on NYSE data
# Build loaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainL, valL, testL, input_dim, L = build_seq2seq_loaders(
    prefix="cleaned_NYSE", key_in_all="NYSE",
    all_processed_data=all_processed_data, batch_size=64
)

# Build 100k model with n_state fixed to 64
prob_mamba = build_prob_mamba_budget_fixed_n(
    input_dim=input_dim, target=100_000, n_state=64,
    n_layers=1, d_state=64, d_conv=4, expand=2,
    d_feat_min=32, d_feat_max=192, d_feat_step=8, tol_frac=0.03, verbose=True
).to(device)

# Train + record (timed)
t0 = time.perf_counter()
hist = train_prob_mamba_with_history(
    prob_mamba, trainL, valL, device=device, epochs=100, lr=1e-3, print_every=10
)
if torch.cuda.is_available():
    torch.cuda.synchronize()
train_seconds = time.perf_counter() - t0
epochs_used = 100
n_train = len(trainL.dataset)
total_samples = epochs_used * n_train
train_throughput = (total_samples / train_seconds) if train_seconds > 0 else float("nan")
print(f"[Timing • Train] {train_seconds:.2f}s ({train_seconds/60:.2f} min) | "
      f"epochs={epochs_used} | samples={total_samples:,} | throughput={train_throughput:.1f} samples/s")

# Plot Val NLL vs epoch
plot_nll_history(hist, every=10, title="Prob-Mamba (NYSE) • NLL")

# Evaluate
metrics = eval_prob_rmse_qlike_laststep(prob_mamba, testL, device)
nll = metrics.get('nll')
nll_str = f"{nll:.6f}" if isinstance(nll, (int, float)) else "N/A"
print(f"[Prob-Mamba • NYSE] test NLL: {nll_str} | last-step RMSE: {metrics['rmse']:.6f} | QLIKE: {metrics['qlike']:.6f}")

In [ ]:
nll = metrics['nll']
nll_str = f"{nll:.6f}" if nll is not None else "N/A"
print(f"[Prob-Mamba • IXIC] test NLL: {nll_str} | last-step RMSE: {metrics['rmse']:.6f} | QLIKE: {metrics['qlike']:.6f}")

In [ ]:
# Prob_mamba on IXIC data
# Build loaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainL, valL, testL, input_dim, L = build_seq2seq_loaders(
    prefix="cleaned_IXIC", key_in_all="IXIC",
    all_processed_data=all_processed_data, batch_size=64
)

# Build 100k model with n_state fixed to 64
prob_mamba = build_prob_mamba_budget_fixed_n(
    input_dim=input_dim, target=100_000, n_state=64,
    n_layers=1, d_state=64, d_conv=4, expand=2,
    d_feat_min=32, d_feat_max=192, d_feat_step=8, tol_frac=0.03, verbose=True
).to(device)

# Train + record (timed)
t0 = time.perf_counter()
hist = train_prob_mamba_with_history(
    prob_mamba, trainL, valL, device=device, epochs=100, lr=1e-3, print_every=10
)
if torch.cuda.is_available():
    torch.cuda.synchronize()
train_seconds = time.perf_counter() - t0
epochs_used = 100
n_train = len(trainL.dataset)
total_samples = epochs_used * n_train
train_throughput = (total_samples / train_seconds) if train_seconds > 0 else float("nan")
print(f"[Timing • Train] {train_seconds:.2f}s ({train_seconds/60:.2f} min) | "
      f"epochs={epochs_used} | samples={total_samples:,} | throughput={train_throughput:.1f} samples/s")

# Plot Val NLL vs epoch
plot_nll_history(hist, every=10, title="Prob-Mamba (IXIC) • NLL")

# Evaluate
metrics = eval_prob_rmse_qlike_laststep(prob_mamba, testL, device)
nll = metrics.get('nll')
nll_str = f"{nll:.6f}" if isinstance(nll, (int, float)) else "N/A"
print(f"[Prob-Mamba • IXIC] test NLL: {nll_str} | last-step RMSE: {metrics['rmse']:.6f} | QLIKE: {metrics['qlike']:.6f}")

In [ ]:
# Prob_mamba on BTC data
# Build loaders
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainL, valL, testL, input_dim, L = build_seq2seq_loaders(
    prefix="BTC_USDT-5m", key_in_all="USDT-5m",
    all_processed_data=all_processed_data, batch_size=64
)

# Build 100k model with n_state fixed to 16
prob_mamba = build_prob_mamba_budget_fixed_n(
    input_dim=input_dim, target=100_000, n_state=16,
    n_layers=1, d_state=16, d_conv=4, expand=2,
    d_feat_min=16, d_feat_max=64, d_feat_step=8, tol_frac=0.03, verbose=True
).to(device)

# Train + record (timed)
t0 = time.perf_counter()
hist = train_prob_mamba_with_history(
    prob_mamba, trainL, valL, device=device, epochs=100, lr=1e-3, print_every=10
)
if torch.cuda.is_available():
    torch.cuda.synchronize()
train_seconds = time.perf_counter() - t0
epochs_used = 100
n_train = len(trainL.dataset)
total_samples = epochs_used * n_train
train_throughput = (total_samples / train_seconds) if train_seconds > 0 else float("nan")
print(f"[Timing • Train] {train_seconds:.2f}s ({train_seconds/60:.2f} min) | "
      f"epochs={epochs_used} | samples={total_samples:,} | throughput={train_throughput:.1f} samples/s")

# Plot Val NLL vs epoch
plot_nll_history(hist, every=10, title="Prob-Mamba (BTC) • NLL")

# Evaluate
metrics = eval_prob_rmse_qlike_laststep(prob_mamba, testL, device)
nll = metrics.get('nll')
nll_str = f"{nll:.6f}" if isinstance(nll, (int, float)) else "N/A"
print(f"[Prob-Mamba • BTC] test NLL: {nll_str} | last-step RMSE: {metrics['rmse']:.6f} | QLIKE: {metrics['qlike']:.6f}")

In [ ]:
with torch.no_grad():
    prob_mamba.eval()
    metrics = eval_prob_rmse_qlike_laststep(prob_mamba, testL, device)
nll = metrics.get('nll')
nll_str = f"{nll:.6f}" if isinstance(nll, (int, float)) else "N/A"
print(f"[Prob-Mamba • BTC] test NLL: {nll_str} | last-step RMSE: {metrics['rmse']:.6f} | QLIKE: {metrics['qlike']:.6f}")